In [6]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import newton, bisect
from scipy.differentiate import derivative

In [7]:
f = lambda x: x**2 - 3*x + 2
alpha = 2

fi1 = lambda x: (x**2 + 2) / 3
fi2 = lambda x: np.sqrt(3*x - 2)
fi3 = lambda x: 3 - (2 / x)
fi4 = lambda x: (x**2 - 2) / (2*x - 3)

dfi1_2 = derivative(fi1, alpha).df
dfi2_2 = derivative(fi2, alpha).df
dfi3_2 = derivative(fi3, alpha).df
dfi4_2 = derivative(fi4, alpha, initial_step=0.3).df
[dfi1_2, dfi2_2, dfi3_2, dfi4_2]

[np.float64(1.3333333333333286),
 np.float64(0.7499999999936833),
 np.float64(0.4999999999925677),
 np.float64(0.0)]

In [8]:
def test_convergence(x, fi, remaining_iter=10):
    return test_convergence(fi(x), fi, remaining_iter-1) if remaining_iter != 0 else x

[test_convergence(alpha, fi1), test_convergence(alpha, fi2), test_convergence(alpha, fi3), test_convergence(alpha, fi4)]

[2.0, np.float64(2.0), 2.0, 2.0]

In [9]:
eps_k = lambda fi, k: test_convergence(alpha, fi, k)

ra = np.log(eps_k(fi1, 9) / eps_k(fi1, 10)) / np.log(eps_k(fi1, 8) / eps_k(fi1, 9))

C:\Users\zerok\AppData\Local\Temp\ipykernel_20484\4167409767.py:3: RuntimeWarning: invalid value encountered in scalar divide
  ra = np.log(eps_k(fi1, 9) / eps_k(fi1, 10)) / np.log(eps_k(fi1, 8) / eps_k(fi1, 9))
